In [1]:
%load_ext autoreload
%autoreload 2

In [9]:
import torch
import torch.nn as nn
import pickle as pkl
import numpy as np

In [3]:
from torch_geometric.datasets import Planetoid
from torch_geometric.loader import DataLoader

dataset_geo = Planetoid(root='/tmp/Cora', name='Cora')
loader = DataLoader(dataset_geo, batch_size=1, shuffle=True)

In [4]:
from dataset import CoraDataset

dataset = CoraDataset("data/cora")

In [8]:
from run import main

main()

Epoch 1
------------------------------
train_loss: 1.9468
train_accuracy: 0.1429
valid_loss: 1.9434
valid_accuracy: 0.0720
test_loss: 1.9963
test_accuracy: 0.1490

Epoch 2
------------------------------
train_loss: 1.9588
train_accuracy: 0.1429
valid_loss: 1.9882
valid_accuracy: 0.1620
test_loss: 1.9605
test_accuracy: 0.1490

Epoch 3
------------------------------
train_loss: 1.9495
train_accuracy: 0.1429
valid_loss: 1.9553
valid_accuracy: 0.1620
test_loss: 1.9267
test_accuracy: 0.3190

Epoch 4
------------------------------
train_loss: 1.9474
train_accuracy: 0.1500
valid_loss: 1.9291
valid_accuracy: 0.2980
test_loss: 1.9224
test_accuracy: 0.3190

Epoch 5
------------------------------
train_loss: 1.9498
train_accuracy: 0.1429
valid_loss: 1.9259
valid_accuracy: 0.3160
test_loss: 1.9308
test_accuracy: 0.3190

Epoch 6
------------------------------
train_loss: 1.9491
train_accuracy: 0.1429
valid_loss: 1.9334
valid_accuracy: 0.3160
test_loss: 1.9546
test_accuracy: 0.0640

Epoch 7
--------

KeyboardInterrupt: 

In [10]:
from model import GraphAttentionNetwork
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dataset = CoraDataset("data/cora", device=device)
dataloader = DataLoader(dataset)

for features, labels, adjacency, train_mask, valid_mask, test_mask in dataloader:
    break

model = GraphAttentionNetwork(
    in_features=dataset.n_features,
    hidden_units=8,
    n_classes=dataset.n_classes,
    n_heads=[8, 1],
)
model.to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=0.005,
    weight_decay=0.0005,
)

def train(featrues, adjacency, train_mask):
    model.train()
    optimizer.zero_grad()
    out = model(features, adjacency)
    loss = F.cross_entropy(out[train_mask], labels[train_mask])
    loss.backward()
    optimizer.step()


@torch.no_grad()
def test(features, adjacency, valid_mask, test_mask):
    model.eval()
    out, accs = model(features, adjacency), []
    for mask in train_mask, valid_mask, test_mask:
        acc = float((out[mask].argmax(-1) == labels[mask]).sum() / mask.sum())
        accs.append(acc)
    return accs


for epoch in range(1, 201):
    train(features, adjacency, train_mask)
    train_acc, val_acc, test_acc = test(features, adjacency, valid_mask, test_mask)
    print(f'Epoch: {epoch:03d}, Train: {train_acc:.4f}, Val: {val_acc:.4f}, '
          f'Test: {test_acc:.4f}')

IndexError: Dimension out of range (expected to be in range of [-2, 1], but got 2)